In [1]:
import torch
from tokenizer import TokenizerWrapper, download_and_merge_text_files, train_tokenizer,download_file_from_url
from dataset_loader import TextDataset, collate_fn
from gru_model import GRULanguageModel
from train_utils import train_model, evaluate_model

In [2]:
# Define model-specific hyperparameter grids
hyperparams_grid = {
    "gru": [
        {"embed_dim": 128, "hidden_dim": 256, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 256, "hidden_dim": 512, "num_layers": 3, "lr": 5e-4}
    ],
    "lstm": [
        {"embed_dim": 128, "hidden_dim": 256, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 256, "hidden_dim": 512, "num_layers": 3, "lr": 1e-4}
    ],
    "rnn": [
        {"embed_dim": 128, "hidden_dim": 128, "num_layers": 2, "lr": 1e-3}
    ],
    "transformer": [
        {"embed_dim": 256, "num_heads": 4, "num_layers": 2, "lr": 1e-3},
        {"embed_dim": 512, "num_heads": 8, "num_layers": 4, "lr": 5e-4}
    ]
}


In [3]:
from train_utils import train_model, evaluate_model
import pandas as pd
import time

results = []

def run_experiments(model_type, ModelClass, grid, tokenizer, train_loader, test_loader, device):
    for idx, config in enumerate(grid):
        model_id = f"{model_type}_{idx}_{int(time.time())}"
        print(f"\n Training {model_id} with config: {config}")
        
        if model_type == "transformer":
            model = ModelClass(
                vocab_size=VOCAB_SIZE,
                embed_dim=config["embed_dim"],
                num_heads=config["num_heads"],
                num_layers=config["num_layers"],
                pad_token_id=tokenizer.get_pad_id()
            ).to(device)
        else:
            model = ModelClass(
                vocab_size=VOCAB_SIZE,
                embed_dim=config["embed_dim"],
                hidden_dim=config["hidden_dim"],
                num_layers=config["num_layers"],
                pad_token_id=tokenizer.get_pad_id()
            ).to(device)
        
        save_path = f"{model_id}.pt"
        train_model(model, train_loader, test_loader, tokenizer, device, save_path, lr=config["lr"], epochs=50)

        ppl, bleu = evaluate_model(model, save_path, test_loader, tokenizer, device)
        results.append({
            "model_type": model_type,
            "config": config,
            "perplexity": ppl,
            "bleu_score": bleu,
            "model_path": save_path
        })


In [4]:

CORPUS_FILE = "corpus.txt"
TRAIN_FILE = "train.jsonl"
TEST_FILE = "test.jsonl"
TOKENIZER_PREFIX = "bpe_tokenizer"
VOCAB_SIZE = 10000
MAX_SEQ_LEN = 128
BATCH_SIZE = 256
EPOCHS = 50
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Select Model Here ===
#MODEL_TYPE = "transformer"  # Options: 'gru', 'lstm', 'rnn', 'transformer'
#MODEL_SAVE_PATH = f"best_{MODEL_TYPE}_model.pt"

# --- Step 1: Download data & train tokenizer ---
#download_file_from_url(TRAIN_URL, TRAIN_FILE)
#download_file_from_url(TEST_URL, TEST_FILE)
#download_and_merge_text_files(DATA_URL, CORPUS_FILE)
train_tokenizer(CORPUS_FILE, TOKENIZER_PREFIX, vocab_size=VOCAB_SIZE)
tokenizer = TokenizerWrapper(f"{TOKENIZER_PREFIX}.model")
     

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: corpus.txt
  input_format: 
  model_prefix: bpe_tokenizer
  model_type: BPE
  vocab_size: 10000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <bos>
  user_defined_symbols: <eos>
  user_defined_symbols: <pad>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_p

In [5]:
train_dataset = TextDataset(TRAIN_FILE, tokenizer, MAX_SEQ_LEN)
test_dataset = TextDataset(TEST_FILE, tokenizer, MAX_SEQ_LEN)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda b: collate_fn(b, tokenizer.get_pad_id()))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda b: collate_fn(b, tokenizer.get_pad_id()))
     

In [6]:

from gru_model import GRULanguageModel
from lstm_model import LSTMLanguageModel
from rnn_model import RNNLanguageModel
from transformer_model import TransformerLanguageModel

EPOCHS = 30

run_experiments("gru", GRULanguageModel, hyperparams_grid["gru"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("lstm", LSTMLanguageModel, hyperparams_grid["lstm"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("rnn", RNNLanguageModel, hyperparams_grid["rnn"], tokenizer, train_loader, test_loader, DEVICE)
run_experiments("transformer", TransformerLanguageModel, hyperparams_grid["transformer"], tokenizer, train_loader, test_loader, DEVICE)

  


 Training gru_0_1744850565 with config: {'embed_dim': 128, 'hidden_dim': 256, 'num_layers': 2, 'lr': 0.001}


/home/vjosep3/.local/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/50: 100%|██████████| 155/155 [00:08<00:00, 18.06it/s]


Epoch 1 | Train Loss: 6.9670 | Val Loss: 6.6449
 Model saved to gru_0_1744850565.pt


Epoch 2/50: 100%|██████████| 155/155 [00:08<00:00, 19.11it/s]


Epoch 2 | Train Loss: 6.5663 | Val Loss: 6.3638
 Model saved to gru_0_1744850565.pt


...

Epoch 48 | Train Loss: 3.8033 | Val Loss: 4.3971


Epoch 49/50: 100%|██████████| 155/155 [00:39<00:00,  3.92it/s]


Epoch 49 | Train Loss: 3.7997 | Val Loss: 4.3974


Epoch 50/50: 100%|██████████| 155/155 [00:39<00:00,  3.89it/s]


Epoch 50 | Train Loss: 3.7911 | Val Loss: 4.3968
📈 Saved loss curve as: TransformerLanguageModel_loss_curve_20250416_211953.png

 Test Perplexity: 81.1430
 Average BLEU Score: 0.0152


In [8]:
results_df = pd.DataFrame(results)
results_df.to_csv("experiment_results.csv", index=False)
results_df.sort_values(by="perplexity").head(10)

,model_type,config,perplexity,bleu_score,model_path
6,transformer,"{'embed_dim': 512, 'num_heads': 8, 'num_layers...",81.142958,0.015212,transformer_1_1744854218.pt
5,transformer,"{'embed_dim': 256, 'num_heads': 4, 'num_layers...",87.876736,0.012722,transformer_0_1744853436.pt
0,gru,"{'embed_dim': 128, 'hidden_dim': 256, 'num_lay...",94.899851,0.013920,gru_0_1744850565.pt
1,gru,"{'embed_dim': 256, 'hidden_dim': 512, 'num_lay...",96.209134,0.014286,gru_1_1744851030.pt
2,lstm,"{'embed_dim': 128, 'hidden_dim': 256, 'num_lay...",100.709278,0.012821,lstm_0_1744851828.pt
4,rnn,"{'embed_dim': 128, 'hidden_dim': 128, 'num_lay...",132.290910,0.010129,rnn_0_1744853135.pt
3,lstm,"{'embed_dim': 256, 'hidden_dim': 512, 'num_lay...",164.521526,0.009868,lstm_1_1744852306.pt


In [15]:
results_df.sort_values(by="model_type").head(11)

,model_type,config,perplexity,bleu_score,model_path
0,gru,"{'embed_dim': 128, 'hidden_dim': 256, 'num_lay...",94.899851,0.013920,gru_0_1744850565.pt
1,gru,"{'embed_dim': 256, 'hidden_dim': 512, 'num_lay...",96.209134,0.014286,gru_1_1744851030.pt
2,lstm,"{'embed_dim': 128, 'hidden_dim': 256, 'num_lay...",100.709278,0.012821,lstm_0_1744851828.pt
3,lstm,"{'embed_dim': 256, 'hidden_dim': 512, 'num_lay...",164.521526,0.009868,lstm_1_1744852306.pt
4,rnn,"{'embed_dim': 128, 'hidden_dim': 128, 'num_lay...",132.290910,0.010129,rnn_0_1744853135.pt
5,transformer,"{'embed_dim': 256, 'num_heads': 4, 'num_layers...",87.876736,0.012722,transformer_0_1744853436.pt
6,transformer,"{'embed_dim': 512, 'num_heads': 8, 'num_layers...",81.142958,0.015212,transformer_1_1744854218.pt


In [28]:

from transformer_model import TransformerLanguageModel

# Example config — must match the config used during training
config = {
    "embed_dim": 512,
    "num_heads": 8,
    "num_layers": 4
}
MODEL_TYPE='Transformer'
# Rebuild the model
model = TransformerLanguageModel(
    vocab_size=VOCAB_SIZE,
    embed_dim=config["embed_dim"],
    num_heads=config["num_heads"],
    num_layers=config["num_layers"],
    pad_token_id=tokenizer.get_pad_id()
).to(DEVICE)

evaluate_model(model, 'transformer_1_1744854218.pt', test_loader, tokenizer, DEVICE)

# --- Step 5: Sample Generation ---
custom_prompts = [
 
    "What do you prefer — cat or dog?",
    "old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the"
]

print(f"\n--- Generations using {MODEL_TYPE.upper()} ---")
for prompt in custom_prompts:
    output = model.generate(tokenizer, prompt, device=DEVICE, return_continuation_only=True)
    print(f" Prompt    : {prompt}")
    print(f" Generated : {output}")



 Test Perplexity: 81.1430
 Average BLEU Score: 0.0152

--- Generations using TRANSFORMER ---
 Prompt    : What do you prefer — cat or dog?
 Generated : I come here in the wood. You may sleep in that coach, and you will come down, father. He took Cosette gently in his hands, and kissed her, propped himself down, kissed her hand. Cosette kept saying to the child.
 Prompt    : old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the
 Generated : door; he could not open it, and hurried off by the door asking whether, if anything could be; but the who was looking up, I would find it a young fox stretched out in the direction, and should


In [29]:
from gru_model import GRULanguageModel

config = {
    "embed_dim": 128,     # Must match training
    "hidden_dim": 256,    # Must match training
    "num_layers": 2       # Must match training
}

model = GRULanguageModel(
    vocab_size=VOCAB_SIZE,
    embed_dim=config["embed_dim"],
    hidden_dim=config["hidden_dim"],
    num_layers=config["num_layers"],
    pad_token_id=tokenizer.get_pad_id()
).to(DEVICE)

MODEL_TYPE='gru'

evaluate_model(model, 'gru_0_1744850565.pt', test_loader, tokenizer, DEVICE)

# --- Step 5: Sample Generation ---
custom_prompts = [
 
    "What do you prefer — cat or dog?",
    "old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the"
]

print(f"\n--- Generations using {MODEL_TYPE.upper()} ---")
for prompt in custom_prompts:
    output = model.generate(tokenizer, prompt, device=DEVICE, return_continuation_only=True)
    print(f" Prompt    : {prompt}")
    print(f" Generated : {output}")




 Test Perplexity: 94.8999
 Average BLEU Score: 0.0139

--- Generations using GRU ---
 Prompt    : What do you prefer — cat or dog?
 Generated : exclaimed Covent. It is with a dreamet drawn me when he spoke. The doctor shook his head. Where is Madeleine? said Fantine. The light is almost, a car. Marius was now lifted and cared, so Rapurzel
 Prompt    : old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the
 Generated : window; he sang, resolutely. The tree got. Her bows hid so long as we could just lean upon a run. around the air was way juffily, and made nothing of it, a fearful


In [30]:
from lstm_model import LSTMLanguageModel

config = {
    "embed_dim": 128,     # Must match training
    "hidden_dim": 256,    # Must match training
    "num_layers": 2       # Must match training
}

model = LSTMLanguageModel(
    vocab_size=VOCAB_SIZE,
    embed_dim=config["embed_dim"],
    hidden_dim=config["hidden_dim"],
    num_layers=config["num_layers"],
    pad_token_id=tokenizer.get_pad_id()
).to(DEVICE)

MODEL_TYPE='lstm'

evaluate_model(model, 'lstm_0_1744851828.pt', test_loader, tokenizer, DEVICE)

# --- Step 5: Sample Generation ---
custom_prompts = [
 
    "What do you prefer — cat or dog?",
    "old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the"
]

print(f"\n--- Generations using {MODEL_TYPE.upper()} ---")
for prompt in custom_prompts:
    output = model.generate(tokenizer, prompt, device=DEVICE, return_continuation_only=True)
    print(f" Prompt    : {prompt}")
    print(f" Generated : {output}")




 Test Perplexity: 100.7093
 Average BLEU Score: 0.0128

--- Generations using LSTM ---
 Prompt    : What do you prefer — cat or dog?
 Generated : asked Il, said she, why will not, how good? Why if I have, as he was old? Char10. Then ran up to the bandit, but Cleft, M. de And what ⁇ , such to be borne in passage
 Prompt    : old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the
 Generated : throne of king and lamented an account of that must best scorn when her mind would have been himself long even lived so


In [31]:
from rnn_model import RNNLanguageModel

config = {
    "embed_dim": 128,     # Must match training
    "hidden_dim": 128,    # Must match training
    "num_layers": 2       # Must match training
}

model = RNNLanguageModel(
    vocab_size=VOCAB_SIZE,
    embed_dim=config["embed_dim"],
    hidden_dim=config["hidden_dim"],
    num_layers=config["num_layers"],
    pad_token_id=tokenizer.get_pad_id()
).to(DEVICE)

MODEL_TYPE='rnn'

evaluate_model(model, 'rnn_0_1744853135.pt', test_loader, tokenizer, DEVICE)

# --- Step 5: Sample Generation ---
custom_prompts = [
 
    "What do you prefer — cat or dog?",
    "old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the"
]

print(f"\n--- Generations using {MODEL_TYPE.upper()} ---")
for prompt in custom_prompts:
    output = model.generate(tokenizer, prompt, device=DEVICE, return_continuation_only=True)
    print(f" Prompt    : {prompt}")
    print(f" Generated : {output}")


 Test Perplexity: 132.2909
 Average BLEU Score: 0.0101

--- Generations using RNN ---
 Prompt    : What do you prefer — cat or dog?
 Generated : What has since you be rained atlook to these fortune, I would not face what to give me to death. Thritted the second of them. He is. This is me that such a couple of people began faming bare up but their
 Prompt    : old Mr Fox stirred under the bench, and cudgelled all the rabble, and drove them and Mrs Fox out of the house. SECOND STORY When old Mr Fox was dead, the wolf came as a suitor, and knocked at the
 Generated : stones down cried and over the seat violently, and again near him. I will make this assistance, and said the King wouldn ⁇ , this Alice was quite a dadz, taking a red merry to the drive Dorothy handed himself upon him. The
